# 水轮机模型 (Water Turbine Model)

`水轮机 (WaterTurbine)` 类模拟一个水力发电水轮机。它根据通过它的水流量及其两端的水力压头差来计算产生的电能。该模型在水量方面是无状态的，意味着入流等于出流，直到达到水轮机的最大容量。本Jupyter Notebook解释了该模型并展示了如何对其进行仿真。

## 状态变量

水轮机的状态由以下变量描述：

- `outflow` (float): 通过水轮机的水流量 (m³/s)。
- `power` (float): 水轮机产生的电功率 (瓦特)。

## 参数

水轮机的性能特点由以下参数定义：

- `efficiency` (float): 水轮机的转换效率 (一个介于0.0和1.0之间的值)。
- `max_flow_rate` (float): 水轮机可以处理的最大流量 (m³/s)。

## 仿真示例

以下示例模拟了一个在变化的入流和恒定的水头差下运行的水轮机。仿真演示了产生的功率如何随流量变化而变化。结果，包括出流量和功率，将被绘制成图表。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` 和 `numpy` (`pip install matplotlib numpy`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from swp.simulation_identification.physical_objects.water_turbine import WaterTurbine
from swp.core.interfaces import State, Parameters

# 水轮机的初始状态和参数
initial_state = State(outflow=0.0, power=0.0)
parameters = Parameters(
    efficiency=0.85,
    max_flow_rate=50.0 # m³/s
)

# 创建一个WaterTurbine实例
turbine = WaterTurbine(name="hydro_turbine_1", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 600  # 时间步长（秒）(10分钟)
simulation_duration = 3600 * 24  # 1天
num_steps = int(simulation_duration / dt)

# 存储结果用于绘图
history = []

# 定义外部条件
upstream_head = 100.0 # m
downstream_head = 20.0 # m

# 生成一个变化的入流模式（例如，每日需求周期）
time_points = np.arange(num_steps)
inflow_pattern = 25 + 25 * np.sin(2 * np.pi * time_points / (24 * 60 / 10)) # 峰值为 50 m³/s

# 运行仿真循环
for i in range(num_steps):
    inflow = inflow_pattern[i]
    turbine.set_inflow(inflow)
    
    action = {
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = turbine.step(action=action, dt=dt)
    history.append(current_state.copy())

# 打印最终状态
print("最终状态:", turbine.get_state())

# 准备绘图数据
time_in_hours = [i * dt / 3600 for i in range(num_steps)]
outflows = [s['outflow'] for s in history]
power_mw = [s['power'] / 1e6 for s in history] # 转换为兆瓦

# 绘制结果
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_in_hours, outflows, label='出流量 (Outflow)')
ax1.plot(time_in_hours, inflow_pattern, label='入流量 (Inflow)', linestyle='--')
ax1.set_ylabel('流量 (m³/s)')
ax1.set_title('水轮机仿真')
ax1.grid(True)
ax1.legend()

ax2.plot(time_in_hours, power_mw, label='发电量 (Power Generated)', color='green')
ax2.set_xlabel('时间 (小时)')
ax2.set_ylabel('功率 (MW)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
